Este Data Set es el obtenido en el trabajo del parcial

In [1]:
# Permite ajustar la anchura de la parte útil de la libreta (reduce los márgenes)
from IPython.core.display import display, HTML
display(HTML("<style>.container{ width:100% }</style>"))

In [2]:
# Importar las librerías que se utilizarán en vuestro proyecto Capstone
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import json
import math
import pylab as pl
from ast import literal_eval#Para pasara de string a list. Eleimnar los "".
import scipy as scp #Para el test de Shapiro Wilks
import random #Para obtener unos elementos al azar de una lista
import statistics #Para la media de una lista
# warnings.filterwarnings('ignore')

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [4]:

df=pd.read_csv("df_movie_limpio.csv")

In [5]:
df.head(2)#Mostrando unos datos
#Ver al final que elenco esta como una lista. Esto debemos de resolver

,imdbId,adult,budget,homepage,original_language,original_title,overview,popularity,poster_path,release_date,...,generos_desenvuelto,paises_de_produccion_desenvuelto,lenguas_habladas_desenvuelto,coleccion_desenvuelto,movieId,tmdbId,rating,elenco,director,years
0,114709,True,30000000.0,http://toystory.disney.com/toy-story,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,...,"['Animation', 'Comedy', 'Family']",['United States of America'],['English'],Toy Story Collection,1,862.0,3.888157,"['Tom Hanks', 'Tim Allen', 'Don Rickles']",John Lasseter,1995.0
1,113497,True,65000000.0,NaN,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,...,"['Adventure', 'Fantasy', 'Family']",['United States of America'],"['English', 'Français']",NaN,2,8844.0,3.236953,"['Robin Williams', 'Jonathan Hyde', 'Kirsten D...",Joe Johnston,1995.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44634 entries, 0 to 44633
Data columns (total 29 columns):
imdbId                              44634 non-null int64
adult                               44634 non-null bool
budget                              44634 non-null float64
homepage                            7688 non-null object
original_language                   44623 non-null object
original_title                      44634 non-null object
overview                            43699 non-null object
popularity                          44634 non-null float64
poster_path                         44272 non-null object
release_date                        44558 non-null object
revenue                             44634 non-null float64
runtime                             44382 non-null float64
status                              44556 non-null object
tagline                             20068 non-null object
title                               44634 non-null object
video                

# Obejtivo
-Predecir cuánto es que va a ganar una película (variable continua) "revenue".

-Predecir a qué tiempo de género pertenece una película.

-Realizar un agrupamiento por clúster (aprendizaje NO supervisado)

# Prediciendo variable continua "revenue" (ingresos)

Para presdecir una variable continua en 1er lugar tenener que ver cuales son las variables (columnas) que se van a tomar: Algunas de las variables que no aportan a la predicción de revenue son:

-imdbId: ya que es un ordenamiento que da la página.
-homepage: ya que es solo el lugar donde vive la página de la película.

-original_title: Para c/ pelícual este es un valor distinto.

-overview: decir de qué trata c/ película también no aporta.

-poster_path: no aporta la imagen de una película.

-tagline: El lema no aporta ya que solo son palabras.

-title: Por el mismo motivo que original_title.

-video: Noa porta.

-coleccion_desenvuelto

-movieId

-tmdbId
    

Primero a los atributos que están como una lista y que en su interior tienen uno o más elementos, tomaremos el 1ro
que es el más importante, por ejemplo en el género de la película "Toy Story" ['Animation', 'Comedy', 'Family'], solo
seleccionamos la 1ra: Animation y así sucesivamente con los otros atributos. Igual se puede tener un error al hacer esto ya que puede que el género más resaltante no sea el 1ro, pero asumamos que es así. 

In [7]:
colum_que_faltan_abrir=["production_companies_abierto","generos_desenvuelto","paises_de_produccion_desenvuelto",
                       "lenguas_habladas_desenvuelto","elenco"]#Son las columnas que estan como una lista y debemos
                                                                #de extraer el 1er elemento

In [8]:
# m1=[[None]*5]*3#columna, fila;
matrix=[[None]*len(colum_que_faltan_abrir)]*len(df.index)
matrix=np.array(matrix)
# matrix[1,2]=23;

Un ejemplo de cómo es que se extra el 1er elemento de un objeto que en realidad es una lista (para pasara de objeto a lista se usa "literal_eval")

In [9]:
df[colum_que_faltan_abrir[1]]
li_ge=[None]*(len(df.index))
#print(sys.getsizeof(li_ge))#Esto es el peso en bytes,357136
df[colum_que_faltan_abrir[1]][1]#en colum_que_faltan_abrir, en len(df.index)
literal_eval(df[colum_que_faltan_abrir[1]][3])[0]

'Comedy'

In [10]:
for i in range(len(colum_que_faltan_abrir)):#Función que me extrae el 1er elemento de cada lista
    for j in range(len(df.index)):
        try:#Se hace el usao de Try/Except ya que hay valores perdidos
            matrix[j,i]=literal_eval(df[colum_que_faltan_abrir[i]][j])[0]
        except:
            pass

Este ya contiene solo el 1er elemento de cada lista.

In [11]:
df_matrix=pd.DataFrame(matrix)
df_matrix.columns=colum_que_faltan_abrir
df_matrix.head(2)

,production_companies_abierto,generos_desenvuelto,paises_de_produccion_desenvuelto,lenguas_habladas_desenvuelto,elenco
0,Pixar Animation Studios,Animation,United States of America,English,Tom Hanks
1,TriStar Pictures,Adventure,United States of America,English,Robin Williams


In [12]:
df.drop(colum_que_faltan_abrir,axis=1,inplace=True)#Retiro todas esas columnas que están como una lista

In [13]:
df.head(2)

,imdbId,adult,budget,homepage,original_language,original_title,overview,popularity,poster_path,release_date,...,title,video,vote_average,vote_count,coleccion_desenvuelto,movieId,tmdbId,rating,director,years
0,114709,True,30000000.0,http://toystory.disney.com/toy-story,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,...,Toy Story,False,7.7,5415.0,Toy Story Collection,1,862.0,3.888157,John Lasseter,1995.0
1,113497,True,65000000.0,NaN,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,...,Jumanji,False,6.9,2413.0,NaN,2,8844.0,3.236953,Joe Johnston,1995.0


In [14]:
df=pd.concat([df,df_matrix], axis=1)#Ahora concateno con los datos que ya tienen el 1er elemento de cada lista.

Mi df que solo contien elementos y ya no listas.

In [15]:
df.head(2)#Ver su parte final para percarse que ya no es una lista

,imdbId,adult,budget,homepage,original_language,original_title,overview,popularity,poster_path,release_date,...,movieId,tmdbId,rating,director,years,production_companies_abierto,generos_desenvuelto,paises_de_produccion_desenvuelto,lenguas_habladas_desenvuelto,elenco
0,114709,True,30000000.0,http://toystory.disney.com/toy-story,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,...,1,862.0,3.888157,John Lasseter,1995.0,Pixar Animation Studios,Animation,United States of America,English,Tom Hanks
1,113497,True,65000000.0,NaN,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,...,2,8844.0,3.236953,Joe Johnston,1995.0,TriStar Pictures,Adventure,United States of America,English,Robin Williams


De este data frame solo voy a trabajar con unas cuantas columnas (atributos)

In [16]:
df_columns=df.columns
columns_drop=['imdbId','adult','homepage','original_title','overview','poster_path','release_date','tagline',
              'title','video',"movieId","tmdbId","production_companies_abierto","paises_de_produccion_desenvuelto","lenguas_habladas_desenvuelto","coleccion_desenvuelto"]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44634 entries, 0 to 44633
Data columns (total 29 columns):
imdbId                              44634 non-null int64
adult                               44634 non-null bool
budget                              44634 non-null float64
homepage                            7688 non-null object
original_language                   44623 non-null object
original_title                      44634 non-null object
overview                            43699 non-null object
popularity                          44634 non-null float64
poster_path                         44272 non-null object
release_date                        44558 non-null object
revenue                             44634 non-null float64
runtime                             44382 non-null float64
status                              44556 non-null object
tagline                             20068 non-null object
title                               44634 non-null object
video                

Notamos que los tipos no son los correctos. Corrijámoslo.

In [17]:
df=df.astype({'original_language': 'category',"status":'category',"generos_desenvuelto":'category',"elenco":"category","director":"category","coleccion_desenvuelto":"category"})
df["release_date"]=pd.to_datetime(df["release_date"])#POr una extraña razón los tipos fueron modificados.

In [18]:
# df.groupby("coleccion_desenvuelto").size().sort_values()      

In [19]:
df2=df.drop(columns_drop,axis=1)#Retiro las columnas que no me interesan
df2.head()

,budget,original_language,popularity,revenue,runtime,status,vote_average,vote_count,rating,director,years,generos_desenvuelto,elenco
0,30000000.0,en,21.946943,373554033.0,81.0,Released,7.7,5415.0,3.888157,John Lasseter,1995.0,Animation,Tom Hanks
1,65000000.0,en,17.015539,262797249.0,104.0,Released,6.9,2413.0,3.236953,Joe Johnston,1995.0,Adventure,Robin Williams
2,0.0,en,11.712900,0.0,101.0,Released,6.5,92.0,3.175550,Howard Deutch,1995.0,Romance,Walter Matthau
3,16000000.0,en,3.859495,81452156.0,127.0,Released,6.1,34.0,2.875713,Forest Whitaker,1995.0,Comedy,Whitney Houston
4,0.0,en,8.387519,76578911.0,106.0,Released,5.7,173.0,3.079565,Charles Shyer,1995.0,Comedy,Steve Martin


Ahora mi df solo contienen variables del tipo numérico y categórico

In [20]:
df2.info()#

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44634 entries, 0 to 44633
Data columns (total 13 columns):
budget                 44634 non-null float64
original_language      44623 non-null category
popularity             44634 non-null float64
revenue                44634 non-null float64
runtime                44382 non-null float64
status                 44556 non-null category
vote_average           44634 non-null float64
vote_count             44634 non-null float64
rating                 44634 non-null float64
director               43780 non-null category
years                  44558 non-null float64
generos_desenvuelto    42220 non-null category
elenco                 42253 non-null category
dtypes: category(5), float64(8)
memory usage: 4.5 MB


In [21]:
df2=df2.dropna()#Dejamos de trabajr con valores perdidos. Para aplicar los distintos modelos no debemos de tener nulos

In [22]:
(df2!=0).apply(pd.Series.value_counts)#Solo nos interesa trabajar con valores no son son cero. Ya que entrenar mi 
#modelo con valores 0 me llevana  un error. No puedo entrenar a mi modelo con revenue=0 o ingresos=0

# Nos devuelve la cantidad de elementos que son distintos de cero (esto es para el true)

,budget,original_language,popularity,revenue,runtime,status,vote_average,vote_count,rating,director,years,generos_desenvuelto,elenco
False,30976,NaN,21,32286,1050,NaN,1762,1688,NaN,NaN,NaN,NaN,NaN
True,8433,39409.0,39388,7123,38359,39409.0,37647,37721,39409.0,39409.0,39409.0,39409.0,39409.0


In [23]:
elementoLImpio=(df2!=0) &(df2.notnull())#No debe de trabajar con valores 0, ni con vacío
elementoLImpio.all(axis=1);#all es como un and gigante y es por filas
df4=df2[elementoLImpio.all(axis=1)]#Este contiene a los datos sin valores 0 ni nulos

In [24]:
# l1=list(np.linspace(0,5,6))#0,len-1,len
# # df2.iloc[l1]

In [25]:
# random.sample(l1, 3)

In [26]:
# Función que solo me extrae unas cuantas filas de una data frame
def retornaSoloUnasFilas(df,numFilasAExtraer):#No fue necesario el uso de esta funcion
    numFil=len(df.index)
    listCompleta=list(np.linspace(0,numFil-1,numFil))
    listDeUnasCuantasFilas=random.sample(listCompleta,numFilasAExtraer)
    return df.iloc[listDeUnasCuantasFilas]

In [27]:
df3=retornaSoloUnasFilas(df2,5000)

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, cat_features, num_features):
        self.cat_features = cat_features
        self.num_features = num_features
        
        # Valores de las variables categoricas, solo nos interesan los que aparecen en train
        self.categories = {}
        
        # StandardScaler para variables numericas
        self.num_scaler = StandardScaler()

    def fit(self, X, *_):
        # Almacenamos los valores de las variables categoricas sin repeticion ni nan
        for c in self.cat_features:
            self.categories[c] = X[c].dropna().unique().tolist()
    
        # Fit StandardScaler
        self.num_scaler.fit(X[self.num_features])

        return self
    
    def transform(self, X, *_):
        out_X = pd.DataFrame(self.num_scaler.transform(X[self.num_features]), columns=self.num_features)
        
        # Transformamos los valores de cada categoria y los concatenamos al DF de salida
        for c in self.cat_features:
            # Cambiamos las variables object a categorical para que las categorias tengan siempre el mismo orden
            c_df = X[c].astype(pd.api.types.CategoricalDtype(categories=self.categories[c]))
            c_df.reset_index(drop=True, inplace=True)
            
            # One-hot encoding. Utilizamos get_dummies de pandas!
            c_dummies = pd.get_dummies(c_df, prefix=c)
            out_X = pd.concat([out_X, c_dummies], axis=1)
            
        return out_X

Extrayendo mis atributos y la variable a predecir (revenue)

In [29]:
X = df4.drop('revenue',axis=1)#
Y = df4['revenue']#Lo que se va a predecir

Obtengo mi lista de solo categórico y de solo numérico. Recodar que este solo tienen los tipos categórico y numérico

In [30]:
cat_features=[]
num_features=[]
for i in range(len(X.columns)):
    if X[(X.columns)[i]].dtype=="float64":
        num_features.append((X.columns)[i])
    else:
        cat_features.append((X.columns)[i])

Divido mi set entre entrenamiento y test

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [32]:
pipe = Pipeline([('preprocessing', CustomTransformer(cat_features, num_features)), 
                 ('clf', LinearRegression())])
pipe.fit(X_train, Y_train)

Y_predict= pipe.predict(X_test)

Y_predict_train= pipe.predict(X_train)

print("Test: ",mean_absolute_error(Y_predict, Y_test))
print("Train: ",mean_absolute_error(Y_predict_train, Y_train))

print("Test R2: ",r2_score(Y_predict, Y_test))
print("Train R2: ",r2_score(Y_predict_train, Y_train))

Test:  2.009699625156412e+17
Train:  19908257.369625136
Test R2:  -0.0017101335072371437
Train R2:  0.9289802422469526


In [33]:
pipe = Pipeline([('preprocessing', CustomTransformer(cat_features, num_features)), ('clf', RandomForestRegressor(n_estimators=10, min_samples_leaf=3, max_features='auto',
                          n_jobs=-1))])
pipe.fit(X_train, Y_train)

Y_predict= pipe.predict(X_test)

Y_predict_train= pipe.predict(X_train)

print("Test: ",mean_absolute_error(Y_predict, Y_test))
print("Train: ",mean_absolute_error(Y_predict_train, Y_train))

print("Test R2: ",r2_score(Y_predict, Y_test))
print("Train R2: ",r2_score(Y_predict_train, Y_train))

Test:  40177499.07228727
Train:  24283862.6878202
Test R2:  0.6428228531770839
Train R2:  0.8913264679082382


Ahora solo voy a trabajar con los atributos numéricos.

In [34]:
df_solo_numerico=df4[num_features]
df_solo_numerico;

In [35]:
X = df_solo_numerico
Y = df4['revenue']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

## Usando Regresión Lineal

Al usar una regresión lineal solo con los atributos numéricos se obtivo una score de 67%

In [36]:
reg=LinearRegression().fit(X_train, Y_train)
reg.score(X_test, Y_test)

0.6734051797681297

## Regresión Polinomial

Ahora aplicaremos una regresión polinomial a mi df de solo numéricos

In [37]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

poly = PolynomialFeatures(degree=4)

X_train_ = poly.fit_transform(X_train)
X_test_=poly.fit_transform(X_test)

clf = linear_model.LinearRegression()
clf.fit(X_train_, Y_train)

clf.score(X_test_, Y_test)# Al menos con el grado 4 se obtienen un score de 63%

0.6282452000819552

## Haciendo el uso de Cross Validation

In [38]:
# Mi X son es de numéricos.
from sklearn.model_selection import cross_val_score
scores = cross_val_score(LinearRegression(), X, Y, cv=20,scoring="r2")
print("Con el uso de una Regresión Lineal")
print("Mean:",np.mean(scores))
print("Desv. Stand.:",np.std(scores))
scores

Con el uso de una Regresión Lineal
Mean: 0.6772318741982063
Desv. Stand.: 0.135923348202496


array([0.48140137, 0.36134233, 0.64475912, 0.56138504, 0.35556779,
       0.58345179, 0.78405653, 0.79785794, 0.71038378, 0.69353807,
       0.75288298, 0.83850115, 0.73519073, 0.73379028, 0.78693977,
       0.75986072, 0.70623563, 0.76411229, 0.77918767, 0.71419249])

In [39]:
poly = PolynomialFeatures(degree=2)
X_ = poly.fit_transform(X)
scores = cross_val_score(LinearRegression(), X_, Y, cv=5,scoring="r2")
print("Cross Validation con el uso de Regresión Polinomial")
print("Mean:",np.mean(scores))
print("Desv. Stand.:",np.std(scores))
scores

Cross Validation con el uso de Regresión Polinomial
Mean: 0.5112025268909084
Desv. Stand.: 0.363730246827615


array([ 0.5511929 ,  0.68951833,  0.68374178,  0.82663832, -0.1950787 ])

## Haciendo el uso de Árbol de Regresión

In [40]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state=10)
cross_val_score(regressor, X, Y, cv=10)

array([0.31813534, 0.34397686, 0.12789376, 0.70745494, 0.55477942,
       0.70553645, 0.68032855, 0.48224592, 0.57812221, 0.44580374])

# Clasificación

Ahora vamoa a clasificar uno de los atributos. En este caso lo que vamos a querer predecir es a qué tipo de género pertenece una película.


In [41]:
Y=df4.generos_desenvuelto
X=df4.drop("generos_desenvuelto",axis=1)

Nos damo cuenta que al clasificar las películas estas no estan distribuidad igualmente en cantidad. 

In [42]:
Y.value_counts()

Drama              1430
Comedy             1102
Action              665
Horror              314
Documentary         287
Adventure           253
Crime               232
Thriller            152
Romance             118
Fantasy             114
Animation           111
Science Fiction      88
Mystery              73
Music                55
Family               51
War                  47
Western              34
History              31
TV Movie             21
Foreign               5
Name: generos_desenvuelto, dtype: int64

In [43]:
print("La cantidad de tipos de géneros en las películas es:",len(Y.value_counts()))

La cantidad de tipos de géneros en las películas es: 20


In [44]:
def pasaADummiUnaColumna(df,nombreColumna):
    c_dummies = pd.get_dummies(df, prefix=nombreColumna)
    return pd.concat([df, c_dummies], axis=1).drop(nombreColumna,axis=1)

In [45]:
df4.status.value_counts()#Como la mayoria esta en release mejor no lo tomo en cuenta y elimino este atributo

Released           5179
Rumored               2
Post Production       2
Planned               0
In Production         0
Canceled              0
Name: status, dtype: int64

In [46]:
df4=df4.drop("status",axis=1)

In [47]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5183 entries, 0 to 44591
Data columns (total 12 columns):
budget                 5183 non-null float64
original_language      5183 non-null category
popularity             5183 non-null float64
revenue                5183 non-null float64
runtime                5183 non-null float64
vote_average           5183 non-null float64
vote_count             5183 non-null float64
rating                 5183 non-null float64
director               5183 non-null category
years                  5183 non-null float64
generos_desenvuelto    5183 non-null category
elenco                 5183 non-null category
dtypes: category(4), float64(8)
memory usage: 1.9 MB


In [48]:
# original_language,director,elenco
df4=df4.astype({"original_language":"object","director":"object","elenco":"object","generos_desenvuelto":"object"})
df4=df4.astype({"original_language":"category","director":"category","elenco":"category",
                "generos_desenvuelto":"category"})# Hago esto para no tener
#categorías que no tengan ninguna frecuencia. (esto se devio a que solo se esta trabajando con valores limpios)

In [49]:
# len(df4.original_language.value_counts())#40
# len(df4.director.value_counts())#2333
len(df4.elenco.value_counts())#2135
#Se estaría agregando 40+2333+2135 atributos a mi df

2135

In [50]:
# df4.original_language.value_counts()
# df4[df4.original_language=="fa"]

df5=pasaADummiUnaColumna(df4,"original_language")#Demora al pasara a dummie ya que se estan creando varias columnas
df5=pasaADummiUnaColumna(df5,"director")
df5=pasaADummiUnaColumna(df5,"elenco")

In [51]:
df5.shape#Veamos las cantidad de atributo que ahora tienen mi df

(5183, 29308)

In [52]:
X = df5.drop("generos_desenvuelto",axis=1)

In [53]:
X.head(2)

,budget,popularity,revenue,runtime,vote_average,vote_count,rating,years,budget,popularity,...,elenco_Zoe Saldana,elenco_Zoe Sloane,elenco_Zoe Weizenbaum,elenco_Zoey Deutch,elenco_Zooey Deschanel,elenco_Zsófia Psotta,elenco_Zuhab Khan,elenco_Şahan Gökbakar,elenco_Гордан Кичић,elenco_杰森·莫玛
0,30000000.0,21.946943,373554033.0,81.0,7.7,5415.0,3.888157,1995.0,30000000.0,21.946943,...,0,0,0,0,0,0,0,0,0,0
1,65000000.0,17.015539,262797249.0,104.0,6.9,2413.0,3.236953,1995.0,65000000.0,17.015539,...,0,0,0,0,0,0,0,0,0,0


In [54]:
Y = df5.generos_desenvuelto

In [55]:
Y.head()

0    Animation
1    Adventure
3       Comedy
5       Action
8       Action
Name: generos_desenvuelto, dtype: category
Categories (20, object): [Action, Adventure, Animation, Comedy, ..., TV Movie, Thriller, War, Western]

In [56]:
len(Y.value_counts())
Y.value_counts()

Drama              1430
Comedy             1102
Action              665
Horror              314
Documentary         287
Adventure           253
Crime               232
Thriller            152
Romance             118
Fantasy             114
Animation           111
Science Fiction      88
Mystery              73
Music                55
Family               51
War                  47
Western              34
History              31
TV Movie             21
Foreign               5
Name: generos_desenvuelto, dtype: int64

In [57]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

## Regresión Logística

In [58]:
# Definimos una funcion para imprimir las metrica de evaluacion
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def print_metrics(y, pred):#y es la clase verdadera
    accuracy = accuracy_score(y_true=y, y_pred=pred)
    precision = precision_score(y_true=y, y_pred=pred)
    recall = recall_score(y_true=y, y_pred=pred)
    f1 = f1_score(y_true=y, y_pred=pred)

    print('\nAccuracy: {:.4f}  Precision:{:.4f}  Recall:{:.4f}  F1:{:.4f}'.format(accuracy, precision, recall, f1))
    
    # return accuracy, precision, recall, f1

In [59]:
from sklearn.linear_model import LogisticRegression
seed=1234
np.random.seed(seed)
clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial',penalty="l2",C=0.5,max_iter=1000)
#multi_class='ovr'
clf.fit(X_train, Y_train)

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [60]:
clf.score(X_test,Y_test)

0.262379421221865

In [61]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, clf.predict(X_test)))#Para ver precision    recall  f1-score ya que los géneros
#no esta parejamente distribuidos

                 precision    recall  f1-score   support

         Action       0.00      0.00      0.00       197
      Adventure       0.11      0.04      0.06        70
      Animation       0.00      0.00      0.00        25
         Comedy       0.25      0.25      0.25       324
          Crime       0.00      0.00      0.00        63
    Documentary       0.00      0.00      0.00        96
          Drama       0.28      0.75      0.40       435
         Family       0.00      0.00      0.00        14
        Fantasy       0.00      0.00      0.00        39
        Foreign       0.00      0.00      0.00         1
        History       0.00      0.00      0.00         9
         Horror       0.00      0.00      0.00        97
          Music       0.00      0.00      0.00         8
        Mystery       0.00      0.00      0.00        26
        Romance       0.00      0.00      0.00        39
Science Fiction       0.00      0.00      0.00        29
       TV Movie       0.00    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Entonces podemos decir que la regresión logística no es la mejor solución para predecir el tipo de película.
Ahora intentemos con otro clasificador (Decision Tree Classifier)

## Decision Tree Classifier

In [62]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(min_samples_split=4,random_state=0,criterion="entropy")
clf.fit(X_train, Y_train)
clf.score(X_test,Y_test)
#Que salga un 100% es sinónimo de overfitting para la Clasificación del género. Y esto es debido a que el arbol 
#se sigue ramificando hasta tener hojas puras.

1.0

In [63]:
#Veamos qué ocrurre cuando forzamos la máxima profundidad del árbol a un valor
clf = DecisionTreeClassifier(min_samples_split=4,random_state=0,criterion="entropy",max_depth=10)
cross_val_score(clf, X,Y, cv=10)
#Al menos ahora ya no tenenmos un modelo tan complicado que se acopla a los datos y ya no tenemos un score del 100%

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.91320755, 0.91825095, 0.91778203, 0.9193858 , 0.92084942,
       0.92427184, 0.92427184, 0.92592593, 0.92759295, 0.92759295])

# AGRUPAMIENTO

En esta parte no vamos a buscar predecir una variable continua o categórica, en este caso vamos a agrupar las pellículas. Aplicaremos distintos algoritmos.

## Usando KMENAS

Para este caso tenemos que saber muy bien cuál el número idónero de clústeres que se van a tomar. Por ser la función de coste un "codo", tomares valores entre 3, 4 o 5 clústeres.

Para esto solo vamos a trabajar en 1er lugar con los valores numéricos.

Como vamos a trabajar condistancias lo que vamos a hacer es hacer una estarización a los atributos.

In [64]:
from sklearn.cluster import KMeans
col_num=["budget","popularity","revenue","runtime","vote_average","vote_count","rating","years"]
df_num=df4[col_num]#Pasemos a Normalizarla
df_num.head(2)

,budget,popularity,revenue,runtime,vote_average,vote_count,rating,years
0,30000000.0,21.946943,373554033.0,81.0,7.7,5415.0,3.888157,1995.0
1,65000000.0,17.015539,262797249.0,104.0,6.9,2413.0,3.236953,1995.0


In [65]:
def normalizaDF(df):#Funcion para normalizar c/ columna de mi df. También se podría hacer un escalamientos.
    for c in df.columns:
        df[c]=(df[c]-df[c].mean())/df[c].std()
    return df

In [66]:
df_num_norm=normalizaDF(df_num)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [67]:
df_num_norm.head(2)# Ahora ya se encuentran normalizado mis datos.

,budget,popularity,revenue,runtime,vote_average,vote_count,rating,years
0,-0.028623,0.883235,1.690634,-1.361326,1.568977,3.729284,1.304745,-0.281081
1,0.845042,0.525012,1.028023,-0.280186,0.680840,1.337748,0.064456,-0.281081


In [68]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(df_num_norm)
print(kmeans.labels_)
print(kmeans.cluster_centers_)

[0 2 1 ... 2 1 1]
[[ 2.49410573  1.42311912  2.76856291  0.64660102  0.56791384  2.69638141
   0.45606649  0.56451936]
 [-0.05902683 -0.14062119 -0.20620453 -0.43767672 -0.81153477 -0.30537027
  -0.80518097  0.19444911]
 [-0.29363277 -0.06720729 -0.19394942  0.32016831  0.68195245 -0.09080621
   0.69164245 -0.26143872]]


Cuando la cantidad de clúster es 3, la distribución de la cantidad de elementos de cada clúster es dispareja. Notamos que 2 clases estan en igual proporción (clase 1 y 2) y una mucho menos (clase 0).

Cantidad de cada clase en porcentaje:

In [69]:
# (pd.DataFrame(kmeans.labels_))[0].value_counts()# Notamos que hay pocos del tipo 0, veamos qué películas son
(pd.DataFrame(kmeans.labels_))[0].value_counts()/len(df4.index)*100

2    48.176732
1    45.089716
0     6.733552
Name: 0, dtype: float64

In [70]:
df4[kmeans.labels_==0].head()

,budget,original_language,popularity,revenue,runtime,vote_average,vote_count,rating,director,years,generos_desenvuelto,elenco
0,30000000.0,en,21.946943,373554033.0,81.0,7.7,5415.0,3.888157,John Lasseter,1995.0,Animation,Tom Hanks
46,33000000.0,en,18.457430,327311859.0,127.0,8.1,5915.0,4.068128,David Fincher,1995.0,Crime,Brad Pitt
108,72000000.0,en,20.755149,210000000.0,177.0,7.7,3404.0,4.016057,Mel Gibson,1995.0,Action,Mel Gibson
162,90000000.0,en,15.138855,366101666.0,128.0,6.9,2094.0,3.503524,John McTiernan,1995.0,Action,Bruce Willis
205,175000000.0,en,16.885184,264218220.0,135.0,5.9,1017.0,2.862219,Kevin Reynolds,1995.0,Adventure,Kevin Costner


Busquemos qué relación es la que tienen estas películas. Veamos como estan la distribucion de géneros

In [71]:
df4[kmeans.labels_==0].generos_desenvuelto.value_counts().head()

Drama          87
Comedy         65
Action         41
Documentary    27
Adventure      21
Name: generos_desenvuelto, dtype: int64

In [72]:
df4[kmeans.labels_==1].generos_desenvuelto.value_counts().head()

Drama          617
Comedy         505
Action         328
Horror         159
Documentary    147
Name: generos_desenvuelto, dtype: int64

In [73]:
df4[kmeans.labels_==2].generos_desenvuelto.value_counts().head()

Drama        726
Comedy       532
Action       296
Horror       138
Adventure    132
Name: generos_desenvuelto, dtype: int64

Hace un momento nos interesó predecir el revenue, veamos cuánto es en media por cada clúster.

In [74]:
print("Clúster 0:",df4[kmeans.labels_==0].revenue.mean())
print("Clúster 1:",df4[kmeans.labels_==1].revenue.mean())
print("Clúster 2:",df4[kmeans.labels_==2].revenue.mean())

Clúster 0: 553732047.7077364
Clúster 1: 56506140.9118528
Clúster 2: 58528019.5855026


Lo princimal que se observa es que el clúster 0 tiene mayor ingreso (revenue). Es decir que nosotro ingresamos los datos para una nueva película y si esta resulta que está en el clúste 0 podríamos decir que tendría un revenue (ingreso) alto.
    

## Usando Cluster JERÁRQUICO

In [75]:
# Como también se buscan distancias en este algoritmo usaremos la data normalizada.
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(n_clusters=3).fit(df_num_norm)

Veamos la distribución de cada clúster. Estos estan en tanto por ciento

In [76]:
# (pd.DataFrame(clustering.labels_))[0].value_counts()
(pd.DataFrame(clustering.labels_))[0].value_counts()/len(df4.index)*100

0    60.312560
2    35.269149
1     4.418291
Name: 0, dtype: float64

Notamos que tenenmos una clase que tieneun gran porcentaje (el de la clase 0). A diferencia de cuando se usó el algoritmo de KMeans en este caso las clases todas están en distinta proporción, en el anterior caso 2 clases estaban en igual proporción.

Veamos el revenue para estos clúster.

In [77]:
print("Clúster 0:",df4[clustering.labels_==0].revenue.mean())
print("Clúster 1:",df4[clustering.labels_==1].revenue.mean())
print("Clúster 2:",df4[clustering.labels_==2].revenue.mean())

Clúster 0: 79739445.44273832
Clúster 1: 672921923.139738
Clúster 2: 37246812.91739606


En este caso el clúster 1 es el que tienen mayor revenue. Es decir que cuando se haga una predicción lo que se busca es que este caiga en dicho clúster para así tener más ingreso (revenue).

## Usando modelo basado en DENSIDAD

Hasta el momento hemos asumido que los clúster se forma por la cercanía que las distintas observaciones tienen. Ahora pasemos a asumir que los clúteres se forman por la conectividad que tienen las observaciones.

Se ursará el DBSCAN. Como se busca cercaní en puntos tambien usaremos la data normalizada.

In [78]:
from sklearn.cluster import DBSCAN
clustering = DBSCAN(eps=3, min_samples=2).fit(df_num_norm)
pd.DataFrame(clustering.labels_)[0].value_counts()

 0    5156
-1      21
 3       2
 2       2
 1       2
Name: 0, dtype: int64

Para la toma de estos parámetros eps=3 y min_samples=2 se tienen 21 outliers

# Conclusiones

-Es muy importante la limpiza de mis datos para no tener error en mi entrenamiento.

-Cuando se buscó predecir el ingreso (revenue) de una película se obtuvo un score de 67% el cual puede ser mejorado. Pero se enfocó en el uso de todos los modelos.

-Cuando se buscó predecir a qué categoría (género) pertenece una película se llego a que cuando se usó DecisionTreeClassifier se llegó a que el modelo se adecuó perfectamente a mis datos y esto es debido en parte a que se usó el one hot encoding (dummy). Igual se bajó la varianza para hacer mi modelo menos complicado.

-Cuando se hace el agrupamento por clúster llegamos a la conclusión que hay clases en las cuales nos convienen que mi película estén ya que así mi película obtendrá un mayor ingreso (revenue)